In [38]:
import pm4py
from pm4py.objects.ocel.obj import OCEL
from typing import Optional
import pandas as pd

In [39]:
ocel = pm4py.read_ocel2_sqlite("../data/event_logs/socel_hinge.sqlite")

/home/grkmr/.cache/pypoetry/virtualenvs/ocean-backend-6RZTwyJ5-py3.10/lib/python3.10/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:70: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_12_existence_type_ocel_time_obj_type_spec_tables', 'const_14_primary_key_object_event_map_type_tables', 'const_15_primary_key_object_event_tables', 'const_16_primary_key_event_object_table', 'const_17_primary_key_object_object_table', 'const_18_primary_key_event_type_spec_tables', 'const_19_foreign_key_event', 'const_20_foreign_key_object', 'const_21_foreign_key_event_object', 'const_22_foreign_key_object_object', 'const_23_foreign_key_event_type_specific', 'const_24_foreign_key_object_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
/home/grkmr/.cache/pypoetry/virtualenvs/ocean-backend-6RZTwyJ5-py3.10/lib/python3.10/site-packages/pm4py/objects/log/util/da

# Filter

## Events

### Filter by activity name

In [78]:
def filter_by_event_types(ocel:OCEL,event_types:list[str],include:bool=True):
    eventMask = ocel.events[ocel.event_activity].isin(event_types)
    return eventMask if include else ~eventMask

### Filter by time range

In [79]:
def filter_by_time_range(ocel:OCEL,start_time:Optional[str]=None, end_time:Optional[str]=None , include:bool=True):
    if start_time is not None:
        start_time = pd.Timestamp(start_time, tz="UTC")
    if end_time is not None:
        end_time = pd.Timestamp(end_time, tz="UTC")
    
    events_df = ocel.events

    mask = pd.Series([True] * len(events_df), index=events_df.index)
    if start_time is not None:
        mask &= events_df["ocel:timestamp"] >= start_time
    if end_time is not None:
        mask &= events_df["ocel:timestamp"] <= end_time

    return mask if include else ~mask

### Filter by activity frequencies

In [80]:
def filter_by_activity_frequency(ocel:OCEL, threshold:int):
    cumulative_freq = ocel.events.value_counts(ocel.event_activity, normalize=True).cumsum()
    included_activities = cumulative_freq[cumulative_freq <= threshold].index
    return filter_by_event_types(ocel, included_activities)

## Objects

In [81]:
### Filter